In [ ]:
import numpy as np
import importlib
from datetime import datetime
import gc
import json

In [ ]:
from helpers import pre_processing_wlan_utils as preprocess_utils
from helpers import classifier_wlan_spectral_utils as classifier_utils
from helpers import tr_models as tr_models

In [ ]:
# Tensorflow
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Keras API
from tensorflow.keras.callbacks import History, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten, Reshape, Activation, Dropout, Add, TimeDistributed, \
    Multiply, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, AveragePooling1D, AveragePooling2D, BatchNormalization, LSTM
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.model_selection import train_test_split

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

In [ ]:
def create_model(x_train, y_train, x_test, y_test):
    input_shape = (2,5000)
    
    dropout = 0.2
    n_filters = 32
    kernel_size = 8
    dilation_depth = 3
    lr = 0.001
    dense_1 = 128
    dense_2 = 64
    pool_c1d_1 = 20
    pool_c1d_2 = 20
    pool_c1d_3 = 5
    batch_size = 32
    print('dropout: ', dropout)
    print('n_filters: ', n_filters)
    print('kernel_size: ', kernel_size)
    print('lr: ', lr)
    print('dense_1: ', dense_1)
    print('dense_2: ', dense_2)
    print('pool_c1d_1: ', pool_c1d_1)
    print('pool_c1d_2: ', pool_c1d_2)
    print('pool_c1d_3: ', pool_c1d_3)
    print('batch_size: ', batch_size)
    print('dilation_depth: ', dilation_depth)
    
    x = Input(shape=input_shape, name='original_input')
    x_reshaped = Reshape(input_shape + (1,), name='reshaped_input')(x)
    skip_connections = []

    out = Conv2D(n_filters, kernel_size, padding='same', name='conv2d')(x_reshaped)
    out = AveragePooling2D((2,1),padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Reshape((out.shape[2], out.shape[3]), name='conv2d_to_1d')(out)

    out = Conv1D(n_filters, kernel_size, dilation_rate=1, padding='causal', name='dilated_conv_1')(out)
    for i in range(1, dilation_depth + 1):
        print('Adding residual blocks')
        tanh_out = Conv1D(n_filters,
                          kernel_size,
                          dilation_rate=kernel_size ** i,
                          padding='causal',
                          name='dilated_conv_%d_tanh' % (kernel_size ** i),
                          activation='tanh'
                          )(out)
        sigm_out = Conv1D(n_filters,
                          kernel_size,
                          dilation_rate=kernel_size ** i,
                          padding='causal',
                          name='dilated_conv_%d_sigm' % (kernel_size ** i),
                          activation='sigmoid'
                          )(out)
        z = Multiply(name='gated_activation_%d' % (i))([tanh_out, sigm_out])
        skip = Conv1D(n_filters, 1, name='skip_%d' % (i))(z)
        out = Add(name='residual_block_%d' % (i))([skip, out])
        skip_connections.append(skip)     
    
    out = Add(name='skip_connections')(skip_connections)
    out = Activation('relu')(out)

    out = Conv1D(n_filters, kernel_size, padding='same', activation='relu')(out)
    out = AveragePooling1D(pool_c1d_1, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv1D(n_filters, kernel_size,  padding='same' , activation='relu')(out)
    out = AveragePooling1D(pool_c1d_2, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv1D(n_filters, kernel_size, padding='same', activation='relu')(out)
    out = AveragePooling1D(pool_c1d_3, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Flatten()(out)
    out = Dense(dense_1, activation='relu')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Dense(dense_2, activation='relu')(out)
    #out = Dropout(dropout)(out)
    #out = BatchNormalization(axis=-1)(out)

    out = Dense(num_classes, activation='softmax')(out)
    model = Model(x, out)

    # ... model fitting
    now = datetime.now()
    datenow = now.strftime('%d%m%y%H%M%S')
    prefix = str(datenow)+"_app"+"_iq_samples_"+str(num_iq_seq)
    file_prefix = prefix + "_classifier"
    save_dir = './'
    saved = save_dir + file_prefix + ".h5"
    hist = save_dir + file_prefix + "_training_history.csv"
    checkpoints = ModelCheckpoint(filepath=saved, monitor='val_loss', verbose=0, save_best_only=True)
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto')
    callbacks = [checkpoints, early_stop]

    epochs = 50
    opt = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

    model.summary()
        
    result = model.fit(x_train, y_train, shuffle=True, validation_split=0.2, verbose = 2,
                         batch_size=batch_size, epochs=epochs, callbacks=callbacks)

    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
def data():
    from helpers import pre_processing_wlan_utils as preprocess_utils
    print('Getting data for model')
    seed = 42
    task = "app"
    label = preprocess_utils.label_index[task]
    num_classes = preprocess_utils.num_classes[task]
    labels_string = preprocess_utils.labels_string[task]

    #adapting for 7 labels
    num_classes = num_classes-1
    del labels_string[7]
    print("Label id: ", label)
    print("Num classes in that label: ", num_classes)
    print("Labels: ", labels_string)
    num_iq_seq = 8000#[100, 300, 500, 700, 1000, 2000]

    padding = 'post'
    load_model=False
    load_dir='../../models/'
    load_file = '22042020175144_saved_wavenet_classifier'
    dataset_folder = '../../dataset/waveforms/'
    dataset_filename = 'waveforms_SNR_v2_16042020_2G_n_mobile_7_classes_app_balanced.mat'
    Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)
    
    X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, num_iq_seq, padding, scale=False)

    print("Oneshot labels")
    Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)

    print("Performing data splitting")
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    #x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

    x_train, x_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.2)
    print(x_train.shape, x_test.shape)    
                  

    print('Data Returned')
    return x_train, y_train, x_test, y_test

In [ ]:
if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials(), 
                                          notebook_name='app_model_evaluation-hyper')
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

In [ ]:
importlib.reload(tr_models)
importlib.reload(classifier_utils)
importlib.reload(preprocess_utils)

In [ ]:
num_iq_seq = 10000
input_shape = (2,num_iq_seq)

In [ ]:
print('Getting data for model')
seed = 42
task = "app"
label = preprocess_utils.label_index[task]
num_classes = preprocess_utils.num_classes[task]
labels_string = preprocess_utils.labels_string[task]

#adapting for 7 labels
num_classes = num_classes-1
del labels_string[7]
print("Label id: ", label)
print("Num classes in that label: ", num_classes)
print("Labels: ", labels_string)

padding = 'post'
load_model=False
load_dir='../../models/'
load_file = '22042020175144_saved_wavenet_classifier'
dataset_folder = '../../dataset/waveforms/'
dataset_filename='waveforms_SNR_16042020_2G_n_mobile_app_new_app_balanced.mat'
#dataset_filename = 'waveforms_SNR_v2_16042020_2G_n_mobile_7_classes_app_balanced.mat'
Xraw, Yraw = classifier_utils.get_raw_xy_spectrum(dataset_folder,dataset_filename)

In [ ]:
X = classifier_utils.pad_or_trunc_x_and_scale(Xraw, num_iq_seq, padding, scale=False)

print("Oneshot labels")
Y = classifier_utils.get_one_hot_labels(Yraw, num_classes, label)

print("Performing data splitting")
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

#x_train, x_test, y_train, y_test = train_test_split(x_test, y_test, test_size=0.2)
print(x_train.shape, x_test.shape)                     

print('Data Returned')
gc.collect()

In [ ]:
dropout = 0.05
n_filters = 32
kernel_size = 16
dilation_depth = 2
lr = 0.0001
batch_size = 32
num_classes = 7
epochs = 100
pool_c1d_1=10
pool_c1d_2=10
pool_c1d_3=5

In [ ]:
def Wavenet_test():
    x = Input(shape=input_shape, name='original_input')
    x_reshaped = Reshape(input_shape + (1,), name='reshaped_input')(x)
    skip_connections = []

    out = Conv2D(n_filters, kernel_size, strides = (1,1), padding='same', name='conv2d')(x_reshaped)
    #out = MaxPooling2D((2,1),padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv2D(n_filters, kernel_size, strides = (1,1), padding='same', name='conv2d')(x_reshaped)
    #out = MaxPooling2D((2,1),padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv2D(n_filters, kernel_size, strides = (1,1), padding='same', name='conv2d')(x_reshaped)
    out = MaxPooling2D((2,1),padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)


    out = Reshape((out.shape[2], out.shape[3]), name='conv2d_to_1d')(out)

    out = Conv1D(n_filters, kernel_size, dilation_rate=1, padding='causal', name='dilated_conv_1')(out)
    for i in range(1, dilation_depth + 1):
        print('Adding residual blocks')
        tanh_out = Conv1D(n_filters,
                          kernel_size,
                          dilation_rate=kernel_size ** i,
                          padding='causal',
                          name='dilated_conv_%d_tanh' % (kernel_size ** i),
                          activation='tanh'
                          )(out)
        sigm_out = Conv1D(n_filters,
                          kernel_size,
                          dilation_rate=kernel_size ** i,
                          padding='causal',
                          name='dilated_conv_%d_sigm' % (kernel_size ** i),
                          activation='sigmoid'
                          )(out)
        z = Multiply(name='gated_activation_%d' % (i))([tanh_out, sigm_out])
        skip = Conv1D(n_filters, 1, name='skip_%d' % (i))(z)
        out = Add(name='residual_block_%d' % (i))([skip, out])
        skip_connections.append(skip)     

    out = Add(name='skip_connections')(skip_connections)
    out = Activation('relu')(out)

    out = Conv1D(n_filters, kernel_size, padding='causal', activation='relu')(out)
    out = AveragePooling1D(5, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv1D(n_filters, kernel_size,  padding='causal' , activation='relu')(out)
    out = AveragePooling1D(pool_c1d_2, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Conv1D(n_filters, kernel_size, padding='same', activation='relu')(out)
    out = AveragePooling1D(5, padding='same')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Flatten()(out)
    out = Dense(128, activation='relu')(out)
    out = Dropout(dropout)(out)
    out = BatchNormalization(axis=-1)(out)

    out = Dense(32, activation='relu')(out)
    #out = Dropout(dropout)(out)
    #out = BatchNormalization(axis=-1)(out)

    out = Dense(num_classes, activation='softmax')(out)
    model = Model(x, out)


    opt = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=True)

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

    model.summary()

    result = model.fit(x_train, y_train, shuffle=True, validation_split=0.2, verbose = 1, batch_size=batch_size, epochs=epochs)

In [ ]:
X_train_cnn = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
X_test_cnn = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

dropout = 0.3
n_filters = 32
kernel_size = 5
batch_size = 128
lr = 0.001

model = Sequential()
model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same', kernel_initializer="glorot_normal", input_shape=input_shape + (1,)))
model.add(MaxPooling2D((1,2), padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D((1,2), padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D(5, padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(n_filters, kernel_size, activation="relu", padding = 'same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Dropout(dropout))
model.add(BatchNormalization(axis=-1))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(32, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))


opt = tf.keras.optimizers.Adam()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

model.summary()


In [ ]:
result = model.fit(X_train_cnn, y_train, shuffle=True, validation_split=0.2, verbose = 1, batch_size=batch_size, epochs=epochs)